Przed oddaniem zadania upewnij się, że wszystko działa poprawnie.
**Uruchom ponownie kernel** (z paska menu: Kernel$\rightarrow$Restart) a następnie
**wykonaj wszystkie komórki** (z paska menu: Cell$\rightarrow$Run All).

Upewnij się, że wypełniłeś wszystkie pola `TU WPISZ KOD` lub `TU WPISZ ODPOWIEDŹ`, oraz
że podałeś swoje imię i nazwisko poniżej:

In [ ]:
NAME = ""

---

# SimCLR
W poprzednim zeszycie rozważana była estymacja rozkładu z wykorzystaniem NCE. InfoNCE jest jego rozszerzeniem do problemu wieloklasowego i zostało wykorzystane w licznych metodach kontrastowego uczenia samonadzorowanego. W niniejszym zeszycie skupimy się na metodzie `SimCLR` [(Chen et al., 2020)](https://arxiv.org/abs/2002.05709). **Należy się dokładnie zapoznać z ideą i zasadą działania SimCLR** (wykorzystując materiały z wykładu oraz publikacje, bądź inne materiały dostępne w sieci). 


W niniejszym zeszycie należy wykonać zadania z zakresu implementacji metody `SimCLR` oraz badania jej hiperparametrów. Model oraz trenowanie zaimplementowano z wykorzystaniem `pytorch_lightning`. Żeby zrozumieć dobrze implementacje, należy zapoznać się z klasą bazową `SSLBase` i ewaluacją na zadaniu docelowym, która została w tej klasie zaimplementowana.

In [ ]:
%load_ext tensorboard

In [ ]:
import copy

import torch
from lightning_fabric import seed_everything
from lightning.pytorch.loggers import TensorBoardLogger
from lightning import Trainer
from torch import nn, Tensor
from torch.nn import functional as F

from src.data import VisionDatamodule
from src.ssl_base import SSLBase
from src.networks import SmallConvnet, MLP
from src.augmentations import get_default_aug

# Zbiór danych
Do uczenia `SimCLR` wykorzystamy wypróbkowany podzbiór zbioru MNIST. Warto zaznaczyć, że jest to wybór podyktowany tylko i wyłącznie ograniczeniami w zasobach obliczeniowych. W rzeczywistości, aby otrzymać konkluzywne i rzetelne wyniki, należałoby skorzystać co najmniej ze zbioru `CIFAR-10` oraz znacznie większego modelu kodera, np. `ResNet`. Jednak do celów dydaktycznych skorzystamy z mniejszego zbioru oraz odpowiednio mniejszego kodera.

In [ ]:
# define parameters
DATA_DIR = "./data"
OUT_DIR = "./data/output/sim_clr"
BATCH_SIZE = 512
NUM_WORKERS = 0
NUM_SAMPLES_PER_CLASS = 300

In [ ]:
datamodule = VisionDatamodule(
    root_dir=DATA_DIR,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    num_samples_per_class=NUM_SAMPLES_PER_CLASS,
)
datamodule.prepare_data()
datamodule.setup()

# Zadanie 2.1 Implementacja modelu SimCLR (2.0 pkt)
W poniższej komórce znajduje się częściowa implementacja modelu `SimCLR`. Uzupełnij brakujące implementacje funkcji:
* `forward` (1.0 pkt) - implementuje przejście w przód modelu (augmentacje do dwóch widoków, forward widoków przez `encoder` oraz `projector`), zwraca parę embeddingów.
* `info_nce_loss` (1.0 pkt) - funkcja straty modelu `SimCLR`:
$$
\ell_{i, j}=-\log \frac{\exp \left(\operatorname{sim}\left(z_i, z_j\right) / \tau\right)}{\sum_{k=1}^{2 N} \mathbb{1}_{[k \neq i]} \exp \left(\operatorname{sim}\left(z_i, z_k\right) / \tau\right)},
$$
gdzie $\operatorname{sim}$ - podobieńśtwo cosinusowe, $\tau$ - parametr temperatury.

Ponadto:
* Wykorzystuj metody z torch'a, unikaj pythonowych pętli.
* Przeanalizuj dokładnie pozostałe elementy implementacji.
* Uruchom uczenie i przeanalizuj wyniki.

In [ ]:
class SimCLRModel(SSLBase):
    def __init__(
        self,
        learning_rate: float,
        weight_decay: float,
        temp: float,
        out_channels: int = 10,
    ):
        super().__init__(
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            out_channels=out_channels,
        )

        # Initialize online network
        self.encoder = SmallConvnet()
        self.projector = MLP(84, 84, 84, plain_last=True)

        self.aug_1 = get_default_aug()
        self.aug_2 = get_default_aug()

        self.temp = temp

    def forward(self, x: Tensor) -> tuple[Tensor, Tensor]:
        # TU WPISZ KOD
        raise NotImplementedError()
        return z_i, z_j

    def forward_repr(self, x: Tensor) -> Tensor:
        return self.encoder(x)

    def training_step(self, batch: Tensor, batch_idx: int) -> Tensor:
        x, _ = batch
        z_i, z_j = self.forward(x)
        loss = self.info_nce_loss(z_i=z_i, z_j=z_j)

        self.log("train/loss", loss, prog_bar=True)

        return loss

    def info_nce_loss(self, z_i: Tensor, z_j: Tensor) -> Tensor:
        # TU WPISZ KOD
        raise NotImplementedError()

In [ ]:
%tensorboard --logdir "./data/output/sim_clr"

In [ ]:
# define hyperparameters
TEMPERATURE = 0.07
LEARNING_RATE = 5e-4
WEIGHT_DECAY = 1e-4
EPOCHS = 75
ACCELERATOR = "cpu" # change to CUDA, if want to train on GPU

seed_everything(42)
model = SimCLRModel(
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    temp=TEMPERATURE,
)
logger = TensorBoardLogger(save_dir=OUT_DIR, default_hp_metric=False)
trainer = Trainer(
    default_root_dir=OUT_DIR,
    max_epochs=EPOCHS,
    logger=logger,
    accelerator=ACCELERATOR,
    num_sanity_val_steps=0,
    log_every_n_steps=10,
)

trainer.fit(model, datamodule)
trainer.test(model, datamodule)

# Zadanie 2.2 Badanie `batch_size` (1 pkt)
Jako jedną z głównych wad metod kontrastowych podaje się wymaganie na stosowanie dużych batch'y. W tym zadaniu:
* Sprawdź co najmniej 3 wielkości batch'a
* Idelanie będzie wykonać 3-5 powtórzeń dla każdej wartości parametru, ale jeśli ograniczają Cię zasoby wystarczy 1
* Z każdym kolejnym uruchominiem uczenia ustawiaj na nowo seed
* Wynik na zbiorze testowym wylicz metodą `trainer.test(...)`, a na podstawie wyników sporządź wykres
* Co zaobserwowałeś? Wyjaśnij otrzymaną zależność.

In [ ]:
# TU WPISZ KOD
raise NotImplementedError()